# Université Paul Sabatier

M1IAFA - Recherche d'information

**TP 2**

Enseignants : Lynda Tamine et Jesús Lovón

Notebook proposé par : José G. Moreno


2024

---

💡 Penser à développer des scripts et fonctions auxiliaires qui vont permettre réutiliser les commandes récurrentes dans ce TP et les prochains. Ceci aussi vous permettra de garder de bon pratique du code et faciliter vos débogages.

---


### Attention ❗❗ Pour la note du TP :
🚨 *Questions de code* : Remplir le code manquant dans les parties correspondantes (le code commenté bénéficie des meilleures notes).

🚨 *Questions ouvertes* : Ecrivez votre réponse textuelle sous forme de commentaires dans les cellules correspondantes.

🚨 *Laissez vos sorties* pour les cellules où vous écrivez le code. Les sorties vides (notebook ou cellules non exécutées) correspondent à 0 points.

---

# TP 2. Application de la RI pour la liaison référentielle

Une application des méthodes de RI est la liaison référentielle. Cette tâche consiste à trouver le lien entre une portion de texte et la page Wikipédia qui le représentent le mieux. Dans ce cas d’application :
* Comme requêtes nous allons utiliser chaque mot dans une phrase (ou chaque mot important).
* Le corpus sont les pages Wikipédia. Donc, chaque page Wikipédia est un document à retrouver.
* La liste de résultats sont les candidats pour la liaison référentielle.

Considère par exemple la chaîne de caractères « ***Thomas and Mario are strikers playing in Munich*** ». Il existe trois sous-chaînes (***Thomas***, ***Mario*** et ***Munich***) ou mots importants qui sont susceptibles d'être identifiées dans une base de connaissances telle que Wikipédia. Une recherche de ces mots-clés dans les pages Wikipédia produirait de nombreux concepts candidats pour ces sous-chaînes. Donc un bon système doit retrouver comme premier résultats les liens aux pages Wikipédia : [Thomas Müller](https://en.wikipedia.org/wiki/Thomas_M%C3%BCller), [Mario Gómez](https://en.wikipedia.org/wiki/Mario_G%C3%B3mez) (ou Mario Götze) et FC [Bayern Munich](https://en.wikipedia.org/wiki/FC_Bayern_Munich). Cette tâche est particulièrement complexe et les méthodes de recherche d’information sont particulièrement utiles pour la résoudre.

> Plusieurs méthodes de liaison référentielle (Entity Linking en anglais) sont disponibles sur internet. Les plus connus sont :

>	* Spotlight → https://demo.dbpedia-spotlight.org/
>	* Babelfy → http://babelfy.org/
>	* TagMe → https://tagme.d4science.org/tagme/

Étapes à suivre pendant ce TP :
1. Liaison avec le texte
2. Les poids
3. Utilisation de termes
4. Le titre
5. L'outil


## 0. Installation de l’environnement de travail

Installation et configuration de librairies comment le TP précédent.

In [2]:
# déclaration de la variable JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
!export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

In [3]:
#installation de pyterrier avec pip
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-9kff8i1r/python-terrier_3d3d2939d9824d639c6fa4fde840e140
  Running command git clone --filter=blob:none --quiet https://github.com/terrier-org/pyterrier.git /tmp/pip-install-9kff8i1r/python-terrier_3d3d2939d9824d639c6fa4fde840e140
  Resolved https://github.com/terrier-org/pyterrier.git to commit 81f20bda483f6044bda89f717ae863e62bab42e5
  Preparing metadata (setup.py) ... done


In [4]:
#Initialization de JVM
import pyterrier as pt
if not pt.started():
  pt.init()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



### Lecture d'un index existant.

Ajoutez le code pour charger l'index créé sur **text** dans le TP 1.

In [4]:
from google.colab import files

# Sélectionnez le fichier à télécharger depuis Google Drive
uploaded = files.upload()


Saving pd_index2.zip to pd_index2.zip
Saving pd_index.zip to pd_index.zip


In [8]:
!unzip ./pd_index.zip


Archive:  ./pd_index.zip
   creating: pd_index/
  inflating: pd_index/data.lexicon.fsomapid  
  inflating: pd_index/data.lexicon.fsomaphash  
  inflating: pd_index/data.meta-0.fsomapfile  
  inflating: pd_index/data.meta.zdata  
  inflating: pd_index/data.document.fsarrayfile  
  inflating: pd_index/data.properties  
  inflating: pd_index/data.lexicon.fsomapfile  
  inflating: pd_index/data.inverted.bf  
  inflating: pd_index/data.direct.bf  
  inflating: pd_index/data.meta.idx  


In [9]:

pd_indexer = pt.DFIndexer("./pd_index")

In [10]:
indexref2 = pt.autoclass("org.terrier.querying.IndexRef").of(os.path.join("./pd_index", "data.properties"))

# 1) Liaison avec le texte
En utilisant l’indexe du champ **text** de Wikipédia (comme dans le TP1) et la requête *« Thomas and Mario are strikers playing in Munich »* vérifiez si vous obtenez les pages Wikipédia respectives (comme le feraient Spotlight, Babelfy ou TagMe) :
* Thomas Müller
* Mario Gómez ou Mario Götze
* FC Bayern Munich

Voici la requete sur l'index mentionné :

In [11]:
results=pt.BatchRetrieve(indexref2, wmodel="BM25", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich")
results.loc[(results['title'] == 'Thomas Mller') | (results['title'] == 'Mario Gmez') | (results['title'] == 'FC Bayern Munich') ]

,qid,docid,docno,title,url,rank,score,query
35,1,60720,247372,Thomas Mller,https://simple.wikipedia.org/wiki/Thomas%20M%C...,35,18.725876,Thomas and Mario are strikers playing in Munich
95,1,23770,91696,Mario Gmez,https://simple.wikipedia.org/wiki/Mario%20G%C3...,95,17.015187,Thomas and Mario are strikers playing in Munich
463,1,1896,6338,FC Bayern Munich,https://simple.wikipedia.org/wiki/FC%20Bayern%...,463,13.506697,Thomas and Mario are strikers playing in Munich


# 2) Les poids

Utilisez les différentes méthodes de pondération prédéfinies sur Pyterrier (http://terrier.org/docs/v5.2/javadoc/org/terrier/matching/models/package-summary.html) pour voir s’il existe des méthodes que vous donnent de meilleurs résultats.

Choisir 5 méthodes (à vous de décider lesquels) et les comparer.

C’est-à-dire inspectez les 10 premiers résultats et vérifiez si les pages d’intérêt (Thomas Müller, Mario Gómez ou Mario Götze et FC Bayern Munich) font partie des résultats, puis les 20 premiers, et finalement les 50 premiers. Existent-ils de différences importantes ?

In [12]:
dataf = pt.BatchRetrieve(indexref2, wmodel="LGD", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)
dataf.loc[(dataf['title'] == 'Thomas Mller') | (dataf['title'] == 'Mario Gmez') | (dataf['title'] == 'FC Bayern Munich') ]

,qid,docid,docno,title,url,rank,score,query
22,1,23770,91696,Mario Gmez,https://simple.wikipedia.org/wiki/Mario%20G%C3...,22,19.105551,Thomas and Mario are strikers playing in Munich
25,1,60720,247372,Thomas Mller,https://simple.wikipedia.org/wiki/Thomas%20M%C...,25,18.311461,Thomas and Mario are strikers playing in Munich


In [13]:
dataf = pt.BatchRetrieve(indexref2, wmodel="DPH", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)
dataf.loc[(dataf['title'] == 'Thomas Mller') | (dataf['title'] == 'Mario Gmez') | (dataf['title'] == 'FC Bayern') ]

,qid,docid,docno,title,url,rank,score,query
40,1,60720,247372,Thomas Mller,https://simple.wikipedia.org/wiki/Thomas%20M%C...,40,10.116397,Thomas and Mario are strikers playing in Munich


In [14]:
dataf = pt.BatchRetrieve(indexref2, wmodel="DFRee", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)
dataf.loc[(dataf['title'] == 'Thomas Mller') | (dataf['title'] == 'Mario Gmez') | (dataf['title'] == 'FC Bayern')]

,qid,docid,docno,title,url,rank,score,query
22,1,60720,247372,Thomas Mller,https://simple.wikipedia.org/wiki/Thomas%20M%C...,22,23.242646,Thomas and Mario are strikers playing in Munich
26,1,23770,91696,Mario Gmez,https://simple.wikipedia.org/wiki/Mario%20G%C3...,26,22.415637,Thomas and Mario are strikers playing in Munich


In [15]:
dataf = pt.BatchRetrieve(indexref2, wmodel="TF_IDF", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)
dataf.loc[(dataf['title'] == 'Thomas Mller') | (dataf['title'] == 'Mario Gmez') | (dataf['title'] == 'FC Bayern')]

,qid,docid,docno,title,url,rank,score,query
33,1,60720,247372,Thomas Mller,https://simple.wikipedia.org/wiki/Thomas%20M%C...,33,10.519862,Thomas and Mario are strikers playing in Munich


In [16]:
dataf = pt.BatchRetrieve(indexref2, wmodel="DirichletLM", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(500)
dataf.loc[(dataf['title'] == 'Thomas Mller') | (dataf['title'] == 'Mario Gmez') | (dataf['title'] == 'FC Bayern')]


,qid,docid,docno,title,url,rank,score,query
232,1,23770,91696,Mario Gmez,https://simple.wikipedia.org/wiki/Mario%20G%C3...,232,4.920038,Thomas and Mario are strikers playing in Munich
484,1,60720,247372,Thomas Mller,https://simple.wikipedia.org/wiki/Thomas%20M%C...,484,4.277024,Thomas and Mario are strikers playing in Munich


**Réponse :** oui il existe des différence importantes , par exemple dans le model choisi "DirchLetLM" , il faut mettre head(500) pour tomber sur deux résultats . pour le reste , on peut dire qu'il existe une différence mais n'est pas trés importante

# 3) Utilisation de termes

Une stratégie consiste à utiliser individuellement chaque terme de la requête pour voir si vous arrivez à trouver les pages d’intérêt. Un code exemple est donné ci-dessous, cependant, il n'est pas suffisant pour faire la tâche.

Proposer une modification de cette fonction basique (à vous de vous inspirer/inventer des modifications qui améliorent les résultats)

Une fois fini votre code, réalisez la même vérification de résultats que dans le point précédent.


In [17]:
# MODIFIER CETTE FONCTION
def basic_term_search(query, method):
  termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
  print(termToSearch)
  for term, page in zip(termToSearch,['Thomas Mller','Mario Gtze', 'FC Bayern Munich']):
    dfres = pt.BatchRetrieve(indexref2, wmodel=method, metadata=["docno","title","url"]).search(term)[['title','score']]
    print(dfres.head(10))
    print("Position de la page attendue dans les résultats :\n",dfres[dfres['title']==page],"\n\n\n")


query = "Thomas and Mario are strikers playing in Munich"
basic_term_search(query, "BM25")


['Thomas', 'Mario', 'Munich']
                         title      score
0             Thomas (surname)  11.700841
1                 Saint Thomas  11.597729
2           Thomas the Apostle  11.447421
3                Thomas County  11.387148
4           Thomas Wedgwood IV  11.210921
5                 Sarah Thomas  11.152142
6                 Helen Thomas  11.145990
7             Gospel of Thomas  11.121265
8      Thomas White (merchant)  11.026214
9  Mallappally Marthoma Church  11.013052
Position de la page attendue dans les résultats :
 Empty DataFrame
Columns: [title, score]
Index: [] 



                          title      score
0                Princess Peach  16.478378
1        Mario (disambiguation)  16.451796
2  Super Mario (disambiguation)  16.419316
3                         Mario  16.395732
4              Charles Martinet  16.349001
5                    Bowser Jr.  16.214048
6                         Wario  16.177717
7                         Luigi  16.132403
8               

In [19]:
import pandas as pd

def updated_search(query, method, indexref2):
    # Définition des stop words
    stop_words = {'the', 'a', 'is', 'are', 'and', 'or', 'in', 'playing'}

    # Termes de recherche pertinents
    pages_of_interest = {
        "Thomas": "Thomas Mller",
        "Mario": "Mario Gtze",
        "Munich": "FC Bayern Munich"
    }

    # Fonction pour filtrer les stop words
    def filter_stop_words(terms):
        return [term for term in terms if term.lower() not in stop_words]

    # Fonction pour récupérer le contexte des termes en majuscules
    def get_context_terms(terms, term_index, max_context_terms=3):
        context_terms = []
        for i in range(term_index + 1, len(terms)):
            if len(context_terms) < max_context_terms and terms[i].lower() not in stop_words:
                context_terms.append(terms[i])
        if len(context_terms) < max_context_terms:
            for i in range(term_index - 1, -1, -1):
                if len(context_terms) < max_context_terms and terms[i].lower() not in stop_words:
                    context_terms.insert(0, terms[i])
        return context_terms

    # Boucle sur les termes de la requête
    terms = query.split()
    for term_index, term in enumerate(terms):
        if term[0].isupper():  # Si le terme commence par une majuscule
            context_terms = get_context_terms(terms, term_index)
            filtered_terms = filter_stop_words([term] + context_terms)
            search_query = ' '.join(filtered_terms)

            # Recherche dans l'index avec le modèle spécifié
            dfres = pt.BatchRetrieve(indexref2, wmodel=method, metadata=["docno", "title", "url"]).search(search_query)[['title', 'score']]
            print(f"Recherche pour: '{search_query}'")
            print(dfres.head(10))

            # Vérification si la page attendue est dans les résultats
            expected_page = pages_of_interest.get(term)
            if expected_page and dfres['title'].str.contains(expected_page, case=False, regex=False).any():
                print(f"La page attendue '{expected_page}' a été trouvée dans les résultats pour '{search_query}':")
                print(dfres[dfres['title'].str.contains(expected_page, case=False, regex=False)][['title', 'score']])
            else:
                print(f"La page attendue '{expected_page}' n'a pas été trouvée dans les résultats pour '{search_query}'.\n")

# Exemple d'utilisation
query = "Thomas and Mario are strikers playing in Munich"
updated_search(query, "BM25", indexref2)


Recherche pour: 'Thomas Mario strikers Munich'
                    title      score
0  Mario Strikers Charged  30.771667
1    Mario Smash Football  30.306808
2           Audi Cup 2013  29.796582
3             Donkey Kong  28.570535
4          Princess Daisy  27.133114
5           Audi Cup 2015  24.388908
6                   Yoshi  24.164655
7         Mario Balotelli  23.841204
8          Princess Peach  23.706325
9                   Wario  21.020281
La page attendue 'Thomas Mller' a été trouvée dans les résultats pour 'Thomas Mario strikers Munich':
            title      score
115  Thomas Mller  14.511453
Recherche pour: 'Mario Thomas strikers Munich'
                    title      score
0  Mario Strikers Charged  30.771667
1    Mario Smash Football  30.306808
2           Audi Cup 2013  29.796582
3             Donkey Kong  28.570535
4          Princess Daisy  27.133114
5           Audi Cup 2015  24.388908
6                   Yoshi  24.164655
7         Mario Balotelli  23.841204
8     

In [20]:
query = "Thomas and Mario are strikers playing in Munich"
updated_search(query, "LGD", indexref2)

Recherche pour: 'Thomas Mario strikers Munich'
                    title      score
0           Audi Cup 2013  24.896580
1  Mario Strikers Charged  22.097048
2    Mario Smash Football  22.031574
3       Alexander Zickler  21.736342
4          Princess Daisy  21.661749
5             Donkey Kong  21.349481
6          Princess Peach  21.177758
7                   Yoshi  20.648091
8                   Wario  19.931632
9                   Luigi  19.842508
La page attendue 'Thomas Mller' a été trouvée dans les résultats pour 'Thomas Mario strikers Munich':
           title      score
41  Thomas Mller  13.952683
Recherche pour: 'Mario Thomas strikers Munich'
                    title      score
0           Audi Cup 2013  24.896580
1  Mario Strikers Charged  22.097048
2    Mario Smash Football  22.031574
3       Alexander Zickler  21.736342
4          Princess Daisy  21.661749
5             Donkey Kong  21.349481
6          Princess Peach  21.177758
7                   Yoshi  20.648091
8       

In [21]:
query = "Thomas and Mario are strikers playing in Munich"
updated_search(query, "TF_IDF" , indexref2)

Recherche pour: 'Thomas Mario strikers Munich'
                    title      score
0  Mario Strikers Charged  16.803635
1    Mario Smash Football  16.550150
2           Audi Cup 2013  16.306862
3             Donkey Kong  15.602268
4          Princess Daisy  14.818170
5           Audi Cup 2015  13.325713
6                   Yoshi  13.197770
7         Mario Balotelli  13.019591
8          Princess Peach  12.947928
9                   Wario  11.481622
La page attendue 'Thomas Mller' a été trouvée dans les résultats pour 'Thomas Mario strikers Munich':
            title     score
113  Thomas Mller  7.954819
Recherche pour: 'Mario Thomas strikers Munich'
                    title      score
0  Mario Strikers Charged  16.803635
1    Mario Smash Football  16.550150
2           Audi Cup 2013  16.306862
3             Donkey Kong  15.602268
4          Princess Daisy  14.818170
5           Audi Cup 2015  13.325713
6                   Yoshi  13.197770
7         Mario Balotelli  13.019591
8       

In [22]:
query = "Thomas and Mario are strikers playing in Munich"
updated_search(query, "DFRee" , indexref2)

Recherche pour: 'Thomas Mario strikers Munich'
                    title      score
0           Audi Cup 2013  30.605758
1          Princess Daisy  28.121133
2       Alexander Zickler  27.953860
3    Mario Smash Football  27.355360
4  Mario Strikers Charged  26.991397
5             Donkey Kong  26.917751
6                   Yoshi  25.883571
7          Princess Peach  25.405639
8                   Luigi  24.558147
9                   Mario  24.372399
La page attendue 'Thomas Mller' a été trouvée dans les résultats pour 'Thomas Mario strikers Munich':
           title      score
42  Thomas Mller  17.265688
Recherche pour: 'Mario Thomas strikers Munich'
                    title      score
0           Audi Cup 2013  30.605758
1          Princess Daisy  28.121133
2       Alexander Zickler  27.953860
3    Mario Smash Football  27.355360
4  Mario Strikers Charged  26.991397
5             Donkey Kong  26.917751
6                   Yoshi  25.883571
7          Princess Peach  25.405639
8       

In [23]:
query = "Thomas and Mario are strikers playing in Munich"
updated_search(query, "DPH" , indexref2)

Recherche pour: 'Thomas Mario strikers Munich'
                    title      score
0          Princess Daisy  15.657696
1           Audi Cup 2013  14.719834
2             Donkey Kong  14.072727
3    Mario Smash Football  14.070726
4                   Yoshi  13.692394
5  Mario Strikers Charged  13.559637
6       Alexander Zickler  13.064107
7          Princess Peach  13.028596
8                   Mario  12.954417
9                   Luigi  12.868258
La page attendue 'Thomas Mller' a été trouvée dans les résultats pour 'Thomas Mario strikers Munich':
           title     score
68  Thomas Mller  7.461633
Recherche pour: 'Mario Thomas strikers Munich'
                    title      score
0          Princess Daisy  15.657696
1           Audi Cup 2013  14.719834
2             Donkey Kong  14.072727
3    Mario Smash Football  14.070726
4                   Yoshi  13.692394
5  Mario Strikers Charged  13.559637
6       Alexander Zickler  13.064107
7          Princess Peach  13.028596
8         

# 4) Le titre
Maintenant utilisez le champ ***title*** avec la requête complète et avec chaque terme. Expliquez  pourquoi les résultats sont différents.

In [24]:
#passage à l'aure indexer
!rm -rf ./pd_index

In [25]:
!unzip ./pd_index2.zip


Archive:  ./pd_index2.zip
   creating: pd_index/
  inflating: pd_index/data.lexicon.fsomapid  
  inflating: pd_index/data.lexicon.fsomaphash  
  inflating: pd_index/data.meta-0.fsomapfile  
  inflating: pd_index/data.meta.zdata  
  inflating: pd_index/data.document.fsarrayfile  
  inflating: pd_index/data.properties  
  inflating: pd_index/data.lexicon.fsomapfile  
  inflating: pd_index/data.inverted.bf  
  inflating: pd_index/data.direct.bf  
  inflating: pd_index/data.meta.idx  


In [26]:
indexref4 = pt.autoclass("org.terrier.querying.IndexRef").of(os.path.join("/content/pd_index", "data.properties"))
data = pt.BatchRetrieve(indexref4, wmodel="DFRee", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)
data.loc[(dataf['title'] == 'Thomas Mller') | (dataf['title'] == 'Mario Gmez') | (dataf['title'] == 'FC Bayern')]

,qid,docid,docno,title,url,rank,score,query


In [32]:
#   Title index
def basic_term_search_ref4(query, method):
  termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
  print(termToSearch)
  for term, page in zip(termToSearch,['Thomas Mller','Mario Gtze', 'FC Bayern Munich']):
    dfres = pt.BatchRetrieve(indexref4, wmodel=method, metadata=["docno","title","url"]).search(term)[['title','score']]
    print(dfres.head(10))
    print("Position de la page attendue dans les résultats :\n",dfres[dfres['title']==page],"\n\n\n")



query = "Thomas and Mario are strikers playing in Munich"
basic_term_search_ref4(query, "BM25")

['Thomas', 'Mario', 'Munich']
                   title      score
0           B. J. Thomas  11.861736
1            Thomas More  11.861736
2  Thomas Thomas (boxer)  11.491433
3          Thomas Hobbes   9.566962
4       Thomas Jefferson   9.566962
5           Thomas Dolby   9.566962
6           Dylan Thomas   9.566962
7          Thomas Edison   9.566962
8        Clarence Thomas   9.566962
9     Thomas the Apostle   9.566962
Position de la page attendue dans les résultats :
            title     score
57  Thomas Mller  9.566962 



20:50:35.837 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
                    title      score
0                   Mario  13.130843
1                 Z Mario  13.130843
2            Mario Jeckle  10.590548
3             Mario Bros.  10.590548
4  Mario (disambiguation)  10.590548
5              Mario Gmez  10.590548
6          Mario Capecchi  10.590548
7              Mario Kar

In [30]:
# improved search title
query = "Thomas and Mario are strikers playing in Munich"
updated_search(query, "BM25", indexref4)


Recherche pour: 'Thomas Mario strikers Munich'
                    title      score
0  Mario Strikers Charged  22.329090
1                  Munich  16.842390
2               Munich RE  16.842390
3            Matt Striker  16.058332
4          Armsel Striker  16.058332
5           Munich U-Bahn  13.584058
6          Munich Airport  13.584058
7       Munich (district)  13.584058
8        Munich Agreement  13.584058
9         Munich massacre  13.584058
La page attendue 'Thomas Mller' a été trouvée dans les résultats pour 'Thomas Mario strikers Munich':
            title     score
152  Thomas Mller  9.566962
Recherche pour: 'Mario Thomas strikers Munich'
                    title      score
0  Mario Strikers Charged  22.329090
1                  Munich  16.842390
2               Munich RE  16.842390
3            Matt Striker  16.058332
4          Armsel Striker  16.058332
5           Munich U-Bahn  13.584058
6          Munich Airport  13.584058
7       Munich (district)  13.584058
8       

**ALL TERMS**

In [33]:
#   tout les termes
def basic_term_search_ref4_all(query, method):
  termToSearch = [x for x in query.split() ]
  print(termToSearch)
  for term, page in zip(termToSearch,['Thomas Mller','Mario Gtze', 'FC Bayern Munich']):
    dfres = pt.BatchRetrieve(indexref4, wmodel=method, metadata=["docno","title","url"]).search(term)[['title','score']]
    print(dfres.head(10))
    print("Position de la page attendue dans les résultats :\n",dfres[dfres['title']==page],"\n\n\n")



query = "Thomas and Mario are strikers playing in Munich"
basic_term_search_ref4_all(query, "BM25")

['Thomas', 'and', 'Mario', 'are', 'strikers', 'playing', 'in', 'Munich']
20:51:03.111 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
                   title      score
0           B. J. Thomas  11.861736
1            Thomas More  11.861736
2  Thomas Thomas (boxer)  11.491433
3          Thomas Hobbes   9.566962
4       Thomas Jefferson   9.566962
5           Thomas Dolby   9.566962
6           Dylan Thomas   9.566962
7          Thomas Edison   9.566962
8        Clarence Thomas   9.566962
9     Thomas the Apostle   9.566962
Position de la page attendue dans les résultats :
            title     score
57  Thomas Mller  9.566962 



20:51:04.765 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
Empty DataFrame
Columns: [title, score]
Index: []
Position de la page attendue dans les résultats :
 Empty DataFrame
Columns: [title

**Réponse:**    Dans la première fonction, seuls les termes commençant par une majuscule sont recherchés ( donc les mots clés de notre query ), Tandis que Dans la deuxième fonction, tous les termes de la requête sont recherchés, y compris les mots de liaison et les termes moins spécifiques ce qui peut élargir la portée de la recherche.

# 5) L'outil
Développez un outil/méthode/système capable de recevoir comme entrée un paragraphe de texte et comme sortie une liste ordonné de pages Wikipédia pour chaque unité lexical (token ou mot) dans le paragraphe.



---



En plus à l’information du TP, vous pouvez utiliser les ressources/idées suivants pour améliorer les résultats du système à développer :
- La combinaison de plusieurs méthodes de pondération c’est possible en utilisant les indications sur https://pyterrier.readthedocs.io/en/latest/pipeline_examples.html .
- Comme mentionné dans le TP1, il existe plusieurs type de champs. On a utilisé celui de la classe TextField. Renseignez vous sur les autres (StringField par exemple) et ses différences.
- Les distances d’édition sont largement utilisés par les moteurs de recherche. Par exemple si vous tapez « paris sain germain » sur Google il va vous suggérer la requête « paris saint germain » l’équipe de football. Vous pouvez utiliser les distances d’édition pour trouver des entités avec une orthographe différent que celle sur Wikipédia. Une de distance très performantes est la distance Levenshtein implémentée dans la classe LuceneLevenshteinDistance du paquet lucene.search.spell.
- Les moteurs de recherche peuvent être utilisés pour calculer la similarité entre les mots ou entre les entités. Par exemple, sur Google si vous tapez « orange » et « carrot » il y a plus de résultats que si vous tapez « purple » et « carrot ». Vous pouvez utiliser cette information pour décider si deux pages Wikipédia font partie d’un même sujet.
- Utilisez le fichier [anchor_text_en.ttl](http://downloads.dbpedia.org/2015-10/core-i18n/en/anchor_text_en.ttl.bz2) qui contient les anchor texts  de la Wikipédia en anglais. Le format du fichier est décrit [ici](https://iai-group.github.io/DBpedia-Entity/index_details.html). Transformez-le dans ce format au autre similaire pour faire l'indexation:

|             Frequency                 	|       Wikipedia Page        	| Anchor Text 	|
|------------------------------	|---------------	|-----------	|
|162| \<Saturn_Award_for_Best_Supporting_Actor> |"Best Supporting Actor"|
|14 |\<André_Golke>| "Golke"|
|13 |\<Charles_Pinckney_National_Historic_Site> |"Charles Pinckney National Historic Site"|
|25 |\<Thomas_Manfredini>| "Manfredini"|
|13 |<Marek_Zieńczuk> |"Marek Zieńczuk"|
|14 |\<Sigurd_Ulveseth>| "Sigurd Ulveseth"|
|104| <Pole_Position_(video_game)>| "Pole Position"|
|10| <Bang_(band)>| "Bang"|
|44| <George_Susce_(catcher)>| "George Susce"|
|11| \<Kyle_Rankin>| "Kyle Rankin"|
|…|…|…|

Donc, la premier colonne correspond au nombre de fois que la chaîne de caractères de la troisième colonne a été utilisé pour faire allusion à la page Wikipédia de la deuxième colonne. C’est-à-dire que "Best Supporting Actor" a été utilisé 162 fois dans Wikipédia pour faire allusion à la page Wikipédia https://en.wikipedia.org/wiki/Saturn_Award_for_Best_Supporting_Actor . Le fichier anchor_text_en.tll  contient les informations de la Wikipédia en anglais et pas de la Wikipédia simple, donc certain titres n’ont pas de correspondance.

In [34]:
def outil(paragraph, model,index) :
  res = []
  mots = paragraph.split()
  mots_majuscules = [mot for mot in mots if mot[0].isupper()]

  for mot in mots_majuscules:
    res.append(pt.BatchRetrieve(index, wmodel=model, metadata=["docno","title","url"]).search(mot).head(10))

  print(res)
text = "Thomas and Mario are strikers playing in Munich"
outil (text,"BM25",indexref4)

20:58:09.879 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
20:58:11.556 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
20:58:12.497 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
[  qid   docid   docno                  title  \
0   1    3543   10625           B. J. Thomas   
1   1   16140   61986            Thomas More   
2   1  107452  466857  Thomas Thomas (boxer)   
3   1    1804    6070          Thomas Hobbes   
4   1    2264    7509       Thomas Jefferson   
5   1    2425    7796           Thomas Dolby   
6   1    3984   12320           Dylan Thomas   
7   1    4160   12950          Thomas Edison   
8   1    6093   19552        Clarence Thomas   
9   1    8167   27207     Thomas the Apostle   

                                   